In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [51]:
def get_slope_range(analyte):
    slope_range_dict = {'Cl':[-55,-35],
                        'K':[48,61],
                        'Ca':[24,34],
                        'pH':[-59,-65]}
    
    return slope_range_dict[analyte]

def get_sensor_dimensions(slope_df, analyte, fluid_name, sensor_type):
    
    slope_range = get_slope_range(analyte)
    
    slope_df = slope_df[slope_df['Analyte']==analyte]
    
    slope_df = slope_df[slope_df['HasSlope']=='Yes']
    
    slope_df_subset = slope_df[(slope_df['Name']==fluid_name)&(slope_df['SensorType']==sensor_type)]
    
    slope_df_subset['Timestamp'] = pd.to_datetime(slope_df_subset['Timestamp'])
    grouped = slope_df_subset.groupby(['POTArray','Analyte','Channel'])
    slope_df_subset['WeekNumber'] = grouped['Timestamp'].transform(lambda x: x.dt.to_period('W').rank(method='dense'))
    
    slope_df_subset = slope_df_subset.groupby(['POTArray','Analyte','Channel','WeekNumber'])['Slope'].median().reset_index()
    
    slope_df_subset['WithinRange'] = slope_df_subset['Slope'].apply(lambda x: True if x>=slope_range[0] and x<=slope_range[1] else False)
    
    slope_df_subset = slope_df_subset.groupby(['POTArray','Channel'])['WithinRange'].agg(list).reset_index()
    
    slope_df_subset['SlopeDecay'] = slope_df_subset.apply(lambda x: 'Ideal' if set(x['WithinRange'])=={True} else ('Include' if set(x['WithinRange'])=={True,False} else 'Exclude'),
                                                          axis=1)
    
    sensor_dimension_df = slope_df.groupby(['POTArray','Channel'])[['L1WetH','L2Rad','Thickness']].mean().reset_index()[['POTArray','Channel','L1WetH','L2Rad','Thickness']]
    
    sensor_dimension_df_processed = pd.merge(slope_df_subset,sensor_dimension_df, on=['POTArray','Channel'])
    
    return sensor_dimension_df_processed

In [52]:
slope_df = pd.read_csv('ViewMEGAResultsDiff.csv')

In [62]:
df = get_sensor_dimensions(slope_df,
                          'Ca',
                          'AQC2',
                          'POT')

In [66]:
df.groupby(['SlopeDecay'])['L1WetH','L2Rad','Thickness'].mean().reset_index()

,SlopeDecay,L1WetH,L2Rad,Thickness
0,Exclude,191.809510,808.099326,70.929677
1,Ideal,155.019000,880.757500,79.233190
2,Include,172.439176,860.970736,73.465122
